In [176]:
import pandas as pd

# Replace 'path_to_file.csv' with the path to your CSV file
csv_file_path = '/Users/jinliu/Desktop/Demo_all.csv'

# Read the CSV file
df = pd.read_csv(csv_file_path)

## Data cleaning
### 1. Remove unnesscary columns
The index column is just a duplicate of the DataFrame index, it could be dropped.
ID related columns are unnesscary for analysis

In [177]:
# List of columns to be removed
columns_to_drop = ['index','documentation_id','ID']

# Drop the unnecessary columns
df = df.drop(columns_to_drop, axis=1)

### 2. Assess Missing Values

Calculate NA percentages for each column and deal with them

In [178]:
# Calculate the percentage of missing values for each column
na_percentages = (df.isnull().sum() / len(df)) * 100

# Print the percentage of NA values for each column
print(na_percentages)


Year                             0.000000
Quarter                          0.000000
CountryOfAsylum                  0.418448
nationality                      3.128073
ethnicity                       65.918185
intentionmove                   22.180047
TotalAdult                       2.492397
TotalMinor                       3.112066
AppliedRefugee                  36.530766
Childinschool                   55.358654
Childvirtualed                  64.527931
disabilityacc_disabilityserv     6.665447
medicalReceived                 54.809869
RiskYes                         31.687742
RiskReturn_medical               1.294217
RiskStay_medical                 6.214986
dtype: float64


CountryOfAsylum missed out some values during the process of data merge

Column like ethnicity and disabilityacc_disabilityserv have a significant number of missing values and 0 and is not crucial for analysis

In [179]:
df = df.drop('ethnicity', axis=1)
df = df.drop('disabilityacc_disabilityserv', axis=1)
df = df.drop('RiskStay_medical', axis=1)

TotalAdult, TotalMinor:
Retain all rows and the data distribution is not heavily skewed, impute missing values using the median.

In [180]:
df['TotalAdult'].fillna(df['TotalAdult'].median(), inplace=True)
df['TotalMinor'].fillna(df['TotalMinor'].median(), inplace=True)
df['RiskReturn_medical'].fillna(df['RiskReturn_medical'].median(), inplace=True)

When totalminor is 0, it means that parents have no children, so later columns that contain NA for education related should be clean. 

For rows where TotalMinor is 0, replace NA in Childvirtualed with 'Not Applicable' (or another appropriate value you choose).

For rows where TotalMinor is greater than 0 and Childvirtualed is NA, decide on an imputation strategy (like replacing with the most frequent value, or a placeholder like 'Unknown').

In [181]:
# Step 1: Replace NA in 'Childvirtualed' with 'Not Applicable' when 'TotalMinor' is 0
df.loc[df['TotalMinor'] == 0, 'Childvirtualed'] = df.loc[df['TotalMinor'] == 0, 'Childvirtualed'].fillna('Not Applicable')

# Step 2: For 'TotalMinor' > 0, replace NA in 'Childvirtualed' with the most frequent value
most_frequent_value = df.loc[df['TotalMinor'] > 0, 'Childvirtualed'].mode()[0]
df.loc[df['TotalMinor'] > 0, 'Childvirtualed'] = df.loc[df['TotalMinor'] > 0, 'Childvirtualed'].fillna(most_frequent_value)

Do the same thing for Childinschool column

In [182]:
df.loc[df['TotalMinor'] == 0, 'Childinschool'] = df.loc[df['TotalMinor'] == 0, 'Childinschool'].fillna('Not Applicable')

most_frequent_value = df.loc[df['TotalMinor'] > 0, 'Childinschool'].mode()[0]
df.loc[df['TotalMinor'] > 0, 'Childinschool'] = df.loc[df['TotalMinor'] > 0, 'Childinschool'].fillna(most_frequent_value)

Impute Based on Other Columns: There is a pattern or correlation with other columns, I use this to infer the missing nationality. Certain nationalities are only found in specific CountryOfAsylum, I use this to impute the values.

In [183]:
# Create a dictionary to map 'CountryOfAsylum' to the most common 'nationality' (in lower case)
asylum_nationality_mapping = df.groupby('CountryOfAsylum')['nationality'].agg(lambda x: x.mode().iat[0].lower()).to_dict()

# Function to apply the mapping
def impute_nationality(row):
    if pd.isna(row['nationality']) and row['CountryOfAsylum'] in asylum_nationality_mapping:
        return asylum_nationality_mapping[row['CountryOfAsylum']]
    else:
        return row['nationality'].lower() if pd.notna(row['nationality']) else row['nationality']

# Apply the function to the 'nationality' column
df['nationality'] = df.apply(impute_nationality, axis=1)

Intentionmove column and AppliedRefugee:
some people answer don'tknow as their answers,we can replace NA with Unknown to show that candidates didn't put answers

In [184]:
# Replace NA in these two columns with 'Unknown'
df['intentionmove'] = df['intentionmove'].fillna('Unknown')
df['AppliedRefugee'] = df['AppliedRefugee'].fillna('Unknown')
df['medicalReceived'] = df['medicalReceived'].fillna('Undecided')
df['RiskYes'] = df['RiskYes'].fillna('Undecided')

In [186]:
# Assuming df is your DataFrame
# Update CountryOfAsylum to "Argentina" where nationality is "venezuela"
df.loc[df['nationality'] == 'venezuela', 'CountryOfAsylum'] = 'Argentina'
print(df)

       Year Quarter CountryOfAsylum nationality intentionmove  TotalAdult  \
0      2020      Q4       Argentina   venezuela          stay         2.0   
1      2020      Q4       Argentina   venezuela      dontknow         3.0   
2      2020      Q4       Argentina   venezuela          stay         2.0   
3      2020      Q4       Argentina   venezuela          stay         3.0   
4      2020      Q4       Argentina   venezuela      relocate         2.0   
...     ...     ...             ...         ...           ...         ...   
43728  2021      Q4       Argentina   venezuela          stay         5.0   
43729  2021      Q4       Argentina   venezuela          stay         1.0   
43730  2021      Q4       Argentina   venezuela          stay         2.0   
43731  2021      Q4       Argentina   venezuela          stay         3.0   
43732  2021      Q4       Argentina   venezuela      relocate         3.0   

       TotalMinor AppliedRefugee   Childinschool     Childvirtualed  \
0   

In [188]:
# Calculate the percentage of missing values for each column
na_percentages = (df.isnull().sum() / len(df)) * 100

# Print the percentage of NA values for each column
print(na_percentages)

Year                  0.0
Quarter               0.0
CountryOfAsylum       0.0
nationality           0.0
intentionmove         0.0
TotalAdult            0.0
TotalMinor            0.0
AppliedRefugee        0.0
Childinschool         0.0
Childvirtualed        0.0
medicalReceived       0.0
RiskYes               0.0
RiskReturn_medical    0.0
dtype: float64


In [187]:
# Display a random 50 rows from the DataFrame with basic styling
styled_df = df.sample(50).style.set_table_styles(
    [{'selector': 'th', 'props': [('font-size', '10pt')]}]
).set_properties(**{'font-size': '9pt', 'border': '1px lightgrey solid !important'}).hide_index()

styled_df

/var/folders/4l/b27mglzj7235f2hz35t8jq9m0000gn/T/ipykernel_94211/200245136.py:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  styled_df = df.sample(50).style.set_table_styles(


Year,Quarter,CountryOfAsylum,nationality,intentionmove,TotalAdult,TotalMinor,AppliedRefugee,Childinschool,Childvirtualed,medicalReceived,RiskYes,RiskReturn_medical
2021,Q1Q2,Guatemala,guatemala,relocate,4.000000,0.000000,Unknown,Not Applicable,Not Applicable,Undecided,Undecided,0.000000
2021,Q3,Guatemala,guatemala,stay,4.000000,2.000000,Unknown,yes,no,Undecided,Undecided,0.000000
2021,Q4,Guatemala,guatemala,stay,1.000000,3.000000,Unknown,yes,yes,Undecided,Undecided,0.000000
2021,Q3,Argentina,venezuela,stay,1.000000,2.000000,yes,no,no,Agree,Disagree,0.000000
2022,Q1,CostaRica,nicaragua,Unknown,1.000000,0.000000,Unknown,Not Applicable,Not Applicable,Undecided,Undecided,0.000000
2021,Q4,Argentina,venezuela,relocate,4.000000,1.000000,no,yes,no,Undecided,StronglyAgree,0.000000
2021,Q3,Argentina,venezuela,relocate,1.000000,3.000000,yes,no,no,StronglyAgree,StronglyAgree,0.000000
2022,Q1,CostaRica,other,Unknown,2.000000,0.000000,Unknown,Not Applicable,Not Applicable,Undecided,Undecided,0.000000
2021,Q4,Argentina,venezuela,relocate,3.000000,1.000000,no,yes,Nainpersonclasses,Agree,Disagree,0.000000
2022,Q1,CostaRica,nicaragua,Unknown,2.000000,1.000000,Unknown,yes,no,Undecided,Undecided,0.000000
